# Feed_forward_Neural_Network_MNIST_pytorch_course

## import the needed packages

In [1]:
import torch
import torch.nn as nn
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torch.autograd import Variable

## Preparing our data

In [2]:
input_size = 784  # Number of input neurons (image pixels)
hidden_size = 400 # Number of hidden neurons
out_size = 10     # Number of classes (0-9)
epochs = 10       # How many times we pass our entire dataset into our network
batch_size = 100  # Input size of the data during one iteration
learning_rate = 0.001 # How fast we are learning

In [3]:
train_dataset = datasets.MNIST(root='./data',
                              train=True,
                              transform=transforms.ToTensor(),
                              download=True)

test_dataset = datasets.MNIST(root='./data',
                             train=False,
                             transform=transforms.ToTensor())

100%|██████████| 9912422/9912422 [00:00<00:00, 101688156.55it/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw



100%|██████████| 28881/28881 [00:00<00:00, 44881694.64it/s]

Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 26786573.18it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 4542/4542 [00:00<00:00, 7435803.58it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [4]:
# Make data iterable by loading it to a loader, shuffle the training data to make it independent of the order
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                         batch_size=batch_size,
                                         shuffle=False)

## Build the NN Model

In [5]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, out_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, out_size)
        
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.relu(out)
        out = self.fc3(out)
        return out

In [6]:
# Create an object of the class, which represents our network
net = Net(input_size, hidden_size, out_size)
CUDA = torch.cuda.is_available()
if CUDA:
    net = net.cuda()
# The loss function. the Cross Entropy loss comes along with softmax. Therefore, no need to specify softmax as well
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

In [7]:
net.parameters

<bound method Module.parameters of Net(
  (fc1): Linear(in_features=784, out_features=400, bias=True)
  (relu): ReLU()
  (fc2): Linear(in_features=400, out_features=400, bias=True)
  (fc3): Linear(in_features=400, out_features=10, bias=True)
)>

## Train and testing the model

In [8]:
# Visulaize the train loader
for i, (images, labels) in enumerate(train_loader):
    print(images.size())
    images = images.view(-1,784)
    print(images.size())

torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([100, 1, 28, 28])
torch.Size([100, 784])
torch.Size([

In [9]:
# Train the network
correct_train = 0
total_train = 0
for epoch in range(epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Flatten the image from size (batch, 1, 28, 28) --> (100, 1, 28, 28) where 1 represents the number of channels (grayscale ---> 1)
        # to size (100, 784) and wrape it in a variable
        images = Variable(images.view(-1, 28*28))
        labels = Variable(labels)
        if CUDA:
            images = images.cuda()
            labels = labels.cuda()
        # Clear the param_grad in param = param - lr*param_grad, so it won't be accumulated
        optimizer.zero_grad()
        outputs = net(images)          # Forward pass
        _, predicted = torch.max(outputs.data, 1)  # Return the second argument of torch.max which represents
                                                   # the index location of each maximum value found
        total_train += labels.size(0)
        if CUDA: 
            correct_train += (predicted.cpu() == labels.cpu()).sum()
        else:
            correct_train += (predicted == labels).sum()
                
        loss = criterion(outputs, labels)          # Difference between the actual and predicted (loss function)
        loss.backward()                            # Backpropagation
        optimizer.step()                           # Update the weights
            
        if (i+1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], Iteration [{i+1}/{len(train_dataset)//batch_size}], Training Loss: {loss.data}, Training Accuracy; {100*correct_train/total_train}%')
print("Done Training ")        

Epoch [1/10], Iteration [100/600], Training Loss: 0.2666783928871155, Training Accuracy; 81.20999908447266%
Epoch [1/10], Iteration [200/600], Training Loss: 0.1771402359008789, Training Accuracy; 86.24500274658203%
Epoch [1/10], Iteration [300/600], Training Loss: 0.1763729453086853, Training Accuracy; 88.90666961669922%
Epoch [1/10], Iteration [400/600], Training Loss: 0.21487590670585632, Training Accuracy; 90.40750122070312%
Epoch [1/10], Iteration [500/600], Training Loss: 0.09828896820545197, Training Accuracy; 91.43800354003906%
Epoch [1/10], Iteration [600/600], Training Loss: 0.061572324484586716, Training Accuracy; 92.2300033569336%
Epoch [2/10], Iteration [100/600], Training Loss: 0.07840762287378311, Training Accuracy; 92.89286041259766%
Epoch [2/10], Iteration [200/600], Training Loss: 0.14224772155284882, Training Accuracy; 93.40125274658203%
Epoch [2/10], Iteration [300/600], Training Loss: 0.0966627299785614, Training Accuracy; 93.83110809326172%
Epoch [2/10], Iteration

In [10]:
# Test the network (No loss and weight calculation, no weight update)
correct = 0
total = 0 
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    if CUDA:
        images = images.cuda()
    # For each input (sample/row) in the batch, the output will contain 10 elements
    outputs = net(images)
    # We could also write: predicted = ouputs.data.max(1)[1]
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)      # Increment the total count (100)
    # We can also use: correct += predicted.eq(Labels).sum()
    if CUDA:
        correct += (predicted.cpu() == labels.cpu()).sum()
    else:
        correct += (predicted == labels).sum()
        
print("Final Test Accuracy: %d %%" % (100 * correct/total))

Final Test Accuracy: 97 %
